<a href="https://colab.research.google.com/github/samanthastaheli/cse450_module2/blob/main/Spencer's_Module_2_stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')

In [3]:
campaign.head()
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  campaign        41188 non-null  int64  
 11  pdays           41188 non-null  int64  
 12  previous        41188 non-null  int64  
 13  poutcome        41188 non-null  object 
 14  emp.var.rate    41188 non-null  float64
 15  cons.price.idx  41188 non-null  float64
 16  cons.conf.idx   41188 non-null  float64
 17  euribor3m       41188 non-null 

In [4]:
table = campaign.groupby("y")
table.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
75,41,blue-collar,divorced,basic.4y,unknown,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
83,49,entrepreneur,married,university.degree,unknown,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
88,49,technician,married,basic.9y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
129,41,technician,married,professional.course,unknown,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes
139,45,blue-collar,married,basic.9y,unknown,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,yes


In [6]:
# Model using poutcome, job, and marital
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics


campaign_drop_na = campaign.dropna()

features = [ "poutcome", "previous"]


X = pd.get_dummies(campaign_drop_na[features], drop_first=True)
y = campaign_drop_na["y"]

# Split our data into training and test data, with 30% reserved for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Build the decision tree
clf = RandomForestClassifier(n_estimators = 100)

# Train it
clf.fit(X_train, y_train)

# Test it 
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp


poutcome_success        0.634920
previous                0.214442
poutcome_nonexistent    0.150637
dtype: float64

In [7]:
y_pred=clf.predict(X_test)

print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          no       0.91      0.99      0.95     10993
         yes       0.63      0.20      0.30      1364

    accuracy                           0.90     12357
   macro avg       0.77      0.59      0.62     12357
weighted avg       0.88      0.90      0.87     12357



Data changes:
change unknowns to na, drop na for each feature
age - bin 
job - one-hot encode???
marital - one-hot encode
education - change to numerical values
default-  no - 0 yes - 1
housing- no - 0 yes - 1
loan- no - 0 yes - 1
contact - one-hot encode
month - one-hot encode or split into different categories
day of the week - one hot encode
pdays - change 999 to -1
poutcome - one-hot encode

In [4]:
#campaign_unknown = campaign.replace('unknown', np.nan)
#campaign_drop_unknown = campaign_drop_unknown.dropna()

ml_education = campaign.education.str.replace("basic.4y", '4').str.replace('basic.6y', '6').str.replace('basic.9y', '9').str.replace('high.school', '12').str.replace('professional.course', '14').str.replace('illiterate', '2').str.replace('university.degree', '16').str.replace('unknown', '0').astype('float')

age_bins = [15, 25, 35, 45, 55, 65, 100]
age_labels = ["younger", "young", "middle", "middlish", "old", "older"]

str(age_labels)

ml_age = pd.cut(campaign['age'], bins=age_bins, labels=age_labels).astype(str)

ml_age = pd.get_dummies(ml_age)

ml_answers = campaign.y.str.replace("no", "0").str.replace("yes", '1').astype('float')

ml_pdays = campaign.pdays.replace(999, -1)

ml_onehot = pd.get_dummies(campaign.filter(['job','month','marital', 'default', 'housing', 'loan', 'contact', 'day_of_week', 'poutcome']))

ml_onehot.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,contact_cellular,contact_telephone,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,1,0
2,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,1,0


In [18]:
final_ml = pd.concat([ml_onehot, 
                         campaign.filter(['campaign', 'previous',
       'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed']),
                         ml_pdays,
                         ml_age,
                         ml_education], axis=1)



final_ml = final_ml.drop("job_student", axis = 1, inplace = False)




,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_technician,job_unemployed,...,euribor3m,nr.employed,pdays,middle,middlish,old,older,young,younger,education
0,0,0,0,1,0,0,0,0,0,0,...,4.857,5191.0,-1,0,0,1,0,0,0,4.0
1,0,0,0,0,0,0,0,1,0,0,...,4.857,5191.0,-1,0,0,1,0,0,0,12.0
2,0,0,0,0,0,0,0,1,0,0,...,4.857,5191.0,-1,1,0,0,0,0,0,12.0
3,1,0,0,0,0,0,0,0,0,0,...,4.857,5191.0,-1,1,0,0,0,0,0,6.0
4,0,0,0,0,0,0,0,1,0,0,...,4.857,5191.0,-1,0,0,1,0,0,0,12.0


In [17]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

final_ml = final_ml.dropna()
campaign_drop_na = campaign.dropna()

#features = [ "poutcome", "previous"]


X = final_ml
y = ml_answers

# Split our data into training and test data, with 20% reserved for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Build the decision tree
clf = RandomForestClassifier(n_estimators = 100)

# Train it
clf.fit(X_train, y_train)

# Test it 
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

euribor3m               0.127732
campaign                0.100540
education               0.085081
nr.employed             0.050911
pdays                   0.034995
cons.conf.idx           0.028565
cons.price.idx          0.026841
poutcome_success        0.026680
housing_yes             0.022482
housing_no              0.022387
job_admin.              0.021487
emp.var.rate            0.021218
young                   0.019346
marital_married         0.018892
middle                  0.018370
marital_single          0.017350
job_technician          0.016991
day_of_week_mon         0.016604
day_of_week_thu         0.015872
day_of_week_tue         0.015813
day_of_week_wed         0.015736
previous                0.015496
middlish                0.015310
day_of_week_fri         0.015139
loan_no                 0.014768
loan_yes                0.014466
job_blue-collar         0.014400
marital_divorced        0.011514
job_management          0.011362
job_services            0.010947
old       